Sno - Серийный номер  
ObservationDate - Дата наблюдения в ММ/ДД/ГГГГ  
Province/State - Провинция или штат наблюдения (может быть пустым, если отсутствует)  
Country/Region - Страна наблюдения  
Last Update - время в формате UTC, когда строка обновляется для данной провинции или страны. (Не стандартизирован и поэтому,   пожалуйста, очистите его перед использованием)  
Confirmed - Совокупное количество подтвержденных случаев до этой даты  
Deaths - Совокупное число смертей до этой даты  
Recovered - Совокупное количество восстановленных случаев до этой даты  

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import json

import dill

import matplotlib.pyplot as plt

from score import score_

import pickle

#import missingno as miss

import seaborn as sns

import math

%matplotlib inline

In [2]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, VotingRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import cross_validate

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, f1_score, mean_squared_error, mean_absolute_error, confusion_matrix, r2_score
from sklearn.manifold import TSNE

from imblearn.over_sampling import SMOTE

In [3]:
# # метрики качества модели
# def score_(y, pred):
#     print('RMSE : ',  math.sqrt(mean_squared_error(y, pred)))
#     print('MAE : ',  mean_absolute_error(y, pred))
#     print('R2 : ', r2_score(y, pred) * 100)

In [4]:
# # переводит дату в читаемый для модели формат
# def date_(df, target):
#     df[target] = df[target].apply(lambda x: str(x).replace('-', '').replace(':', '').replace(' ', ''), 1)
#     df[target] = df[target]. astype(np.int64)
#     return df

In [5]:
# # фун-ция создает новые признаки
# def poly_(df):
#     poly = PolynomialFeatures()
#     poly = pd.DataFrame(poly.fit_transform(df))
# #     poly = poly.rename({1: 'lat', 2: 'long', 3: 'date', 4: 'countryregion', 5: 'deaths', 6: 'recovered', 7: 'active'}, axis=1)
#     return poly

In [6]:
df = pd.read_csv('full_table.csv', index_col=0)

In [7]:
with open('country.json') as json_data:
    country = json.load(json_data)

In [8]:
# df = pd.read_csv('df_all.csv', index_col = 0)

In [9]:
df[:3]

,lat,long,date,provincestate,countryregion,confirmed,deaths,recovered,active
0,33.939110,67.709953,2020-01-22,NaN,Afghanistan,0,0,0.0,0.0
176,4.210484,101.975766,2020-01-22,NaN,Malaysia,0,0,0.0,0.0
177,3.202800,73.220700,2020-01-22,NaN,Maldives,0,0,0.0,0.0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135356 entries, 0 to 136343
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   lat            135356 non-null  float64
 1   long           135356 non-null  float64
 2   date           135356 non-null  object 
 3   provincestate  41496 non-null   object 
 4   countryregion  135356 non-null  object 
 5   confirmed      135356 non-null  int64  
 6   deaths         135356 non-null  int64  
 7   recovered      135356 non-null  float64
 8   active         135356 non-null  float64
dtypes: float64(4), int64(2), object(3)
memory usage: 10.3+ MB


In [11]:
target = 'confirmed'
RANDOM_STATE = 42

del df['provincestate']
del df['lat']
del df['long']
# del df['confirmed']

In [12]:
df['date'] = pd.to_datetime(df['date'])
# df = df.sort_values(by= 'countryregion')

In [13]:
X_train = df[df['date'].dt.year == 2020]
X_test = df[df['date'].dt.year == 2021]

In [14]:
y_train = X_train[target]
y_test = X_test[target]
X_train = X_train.drop(target,1)
X_test = X_test.drop(target,1)

X_train.to_csv('X_train.csv')
X_test.to_csv('X_test.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')

In [15]:
class DateSelector(BaseEstimator, TransformerMixin):

    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        #print('date fit')
        return self
 
    def transform(self, X):
        try:
            tran_tab = str.maketrans(dict.fromkeys(list("\n ,|, . a-  # @ ! $ % ^ & * = ] [ \'  \" < > ~  ) ( № ; : ? +  / -"), ""))
            X[self.key] = X[self.key].apply(lambda x:  str(x).lower().translate(tran_tab)[:8])
    #             X[self.key] = X[self.key].apply(lambda x: str(x).replace('-', '').replace(':', '').replace(' ', ''), 1)
            X[self.key] = X[self.key].astype(np.int64)
            print('step 1 : OK')
    #             return X[self.key].values.reshape((-1,1))
            return  X

        except:
            print('not processed step 1')
            return X[self.key].values.reshape((-1,1))    
        
        
        
        

class FactorizeSelector(BaseEstimator, TransformerMixin):

    def __init__(self, key, country):
        self.key = key
        self.country = country
        
    def fit(self, X, y=None):
        return self
        
        
    def transform(self, X):
        
        try:
            X[self.key] = X[self.key].map(self.country).fillna(0)
            print('step 2 : OK')
#             return self.country 
#             return X[self.key].values.reshape((-1,1))
            return X
        except:
            print('not processed step 2')
            return X[self.key].values.reshape((-1,1))
        

        

class CustomCatBoostRegressor(CatBoostRegressor):

    def fit(self, X, y=None, **fit_params):
        return super().fit(X,y)
       


In [16]:
class ColumnSelector(BaseEstimator, TransformerMixin):

    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        try:
            print(' step 3 : OK')
            return X[[self.column]]
        except:
            print('not processed step ')
            return X[[self.column]]

dates = Pipeline([('date' , DateSelector('date'))
                   ])

countryregion = Pipeline([('countryregion' , FactorizeSelector('countryregion', country))
                   ])



final_transformers = [('datas',dates),('c_r',countryregion)]
poly_features = [  'deaths',  'recovered', 'active']
for poly_feat in poly_features:
    prepare_date = Pipeline([
        ('selector', ColumnSelector(column=poly_feat)),
        ('transformer', PolynomialFeatures())
    ])
    
    final_transformers.append((poly_feat, prepare_date))
    

feats = FeatureUnion(final_transformers, verbose=1)
feature_processing = Pipeline([('feats', feats)])
'''
polys = Pipeline([('poly', PolyTrans_())])

feats = FeatureUnion([('date', dates),
                      ('countryregion', countryregion),
                      ('polys', polys),
                      
                      ])
'''                      

"\npolys = Pipeline([('poly', PolyTrans_())])\n\nfeats = FeatureUnion([('date', dates),\n                      ('countryregion', countryregion),\n                      ('polys', polys),\n                      \n                      ])\n"

In [17]:
param_model = VotingRegressor(
         estimators = [
             ('lr', LinearRegression()),
             ('forest', RandomForestRegressor(n_estimators=10, random_state=42)),
             ('cat', CatBoostRegressor(random_state = 42, silent=True, depth = 3, learning_rate =  0.04))])

In [18]:
%%time

pipeline = Pipeline([
     ('features', feats),
     ('param_model', param_model),
#      ('regressor', CatBoostRegressor(random_state = 42, silent=True, depth = 3, learning_rate =  0.04)),
#     ('classifier', LogisticRegression()),
], verbose = 1)

#pipeline.transform(X_test)
# pipeline.fit(X_train,np.array(y_train).T)
pipeline.fit(X_train,y_train)

step 1 : OK
[FeatureUnion] ......... (step 1 of 5) Processing datas, total=   1.1s
step 2 : OK
[FeatureUnion] ........... (step 2 of 5) Processing c_r, total=   0.0s
 step 3 : OK
[FeatureUnion] ........ (step 3 of 5) Processing deaths, total=   0.0s
 step 3 : OK
[FeatureUnion] ..... (step 4 of 5) Processing recovered, total=   0.0s
 step 3 : OK
[FeatureUnion] ........ (step 5 of 5) Processing active, total=   0.0s
[Pipeline] .......... (step 1 of 2) Processing features, total=   1.1s
[Pipeline] ....... (step 2 of 2) Processing param_model, total=  20.0s
Wall time: 21.2 s


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('datas',
                                                 Pipeline(steps=[('date',
                                                                  DateSelector(key='date'))])),
                                                ('c_r',
                                                 Pipeline(steps=[('countryregion',
                                                                  FactorizeSelector(country={'Afghanistan': 1,
                                                                                             'Albania': 2,
                                                                                             'Algeria': 3,
                                                                                             'Andorra': 4,
                                                                                             'Angola': 5,
                                                                  

In [19]:
y_pred = pipeline.predict(X_test)

step 1 : OK
step 2 : OK
 step 3 : OK
 step 3 : OK
 step 3 : OK


In [20]:
score_(y_test, y_pred)

RMSE :  517237.26685388916
MAE :  68775.2297239963
R2 :  94.4567080735123


# Save Model

In [21]:
# with open("Voronkov_pipeline_model.pkl", "wb") as f:
#         pickle.dump(pipeline, f)

In [22]:
with open("Voronkov_pipeline_model.dill", "wb") as f:
    dill.dump(pipeline, f)

# Loading model

In [23]:
# with open("Voronkov_pipeline_model.pkl", "rb") as f:
#     Voronkov_model = pickle.load(f)

In [24]:
with open('Voronkov_pipeline_model.dill', 'rb') as in_strm:
    Voronkov_model = dill.load(in_strm)

In [25]:
X_test = pd.read_csv('X_test.csv', index_col = 0)
y_test = pd.read_csv('y_test.csv', index_col = 0)

In [26]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40826 entries, 95402 to 136343
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           40826 non-null  object 
 1   countryregion  40826 non-null  object 
 2   deaths         40826 non-null  int64  
 3   recovered      40826 non-null  float64
 4   active         40826 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.9+ MB


In [27]:
pred = Voronkov_model.predict(X_test)

step 1 : OK
step 2 : OK
 step 3 : OK
 step 3 : OK
 step 3 : OK


In [28]:
score_(y_test, pred)

RMSE :  517237.26685388916
MAE :  68775.2297239963
R2 :  94.4567080735123
